# Обучение с подкрепление тестирование

**Выполнил Кузин Мирослав**

In [1]:
from mancala import Kalah
import torch

**Болваны**

In [2]:
# Есть ошибки, исправить
def do_simple_bot_step(state: torch.Tensor) -> int:
    nonzero_state_indexs = torch.nonzero(state[:6]).flatten()
    rez = nonzero_state_indexs[torch.randint(0, len(nonzero_state_indexs), (1,))[0]]
    return rez + 1


# Бот МАСТЕР  захвата
def do_prof_bot_step(state: torch.Tensor) -> int:
    nonzero_state_indexs = torch.nonzero(state[:6]).flatten()
    state_copy = state.clone()
    captured = []
    additional_moves = []
    for cursor in nonzero_state_indexs:
        cursor = int(cursor)
        n = int(cursor)
        temp = int(state_copy[cursor])
        state_copy[cursor] = 0
        while temp > 0:
            temp -= 1
            if cursor in range(0, 6):
                cursor += 1
            elif cursor == 6:
                cursor = 13
            elif cursor in range(9, 14):
                cursor -= 1
            elif cursor == 8:
                cursor = 0

            state_copy[cursor] += 1

        if cursor == 6:
            additional_moves.append(n)

        if cursor < 6 and state_copy[cursor] == 1 and state_copy[cursor%6-6] != 0:
            state_copy[6] += sum([state_copy[cursor], state_copy[cursor%6-6]])
            captured.append((n, sum([state_copy[cursor], state_copy[cursor%6-6]])))
        state_copy = state.clone()


    if captured != []:
        rez_ind = sorted(captured, key=lambda x: x[1])
        rez = rez_ind[-1][0]
    elif additional_moves != []:
        rez = additional_moves[torch.randint(0, len(additional_moves), (1,))[0]]
    else:
        rez = nonzero_state_indexs[torch.randint(0, len(nonzero_state_indexs), (1,))[0]]
    return rez + 1


do_simple_bot_step(torch.Tensor([1, 0, 1, 0, 0, 0, 0]))
print(do_prof_bot_step(torch.Tensor([6, 6, 6, 6, 6, 6, 0, 0, 6, 6, 6, 6, 6, 6])))

1


**Модель**

In [3]:
model = torch.load("Kuzin_M_A_4.6_model2_winner_test_2_10_float.pt")

In [4]:
model

Sequential(
  (0): Linear(in_features=14, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=24, bias=True)
  (3): ReLU()
  (4): Linear(in_features=24, out_features=6, bias=True)
  (5): Softmax(dim=None)
)

**Настройка тестирования**

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


**Обучение**

In [5]:
# Проверка работы модели
game = Kalah()
# model.to(device)
model.to(torch.float)

print(game.get_general_state())
model(game.get_state().to(torch.float))
# tensor([1.4313e-03, 9.9306e-01, 5.4998e-03, 6.9316e-06, 2.3116e-06, 2.4980e-08],
#        grad_fn=<SoftmaxBackward0>)

tensor([6, 6, 6, 6, 6, 6, 0, 0, 6, 6, 6, 6, 6, 6])


tensor([1.9094e-07, 7.8492e-01, 2.1507e-01, 1.0223e-05, 6.6190e-10, 8.4509e-11],
       grad_fn=<SoftmaxBackward0>)

*Награды и штрафы*

In [6]:
# Второй этап обучения
winner_reward_stage_2 = 10
loser_reward_stage_2 = -10
draw_reward_stage_2 = -5
bad_step_reward_stage_2 = -0.5 # общий
good_step_reward_stage_2 = 0.5
good_step_captured_reward_stage_1 = 1

# Первый этап обучения
winner_reward_stage_1 = 1e-3
loser_reward_stage_1 = -1e-3
draw_reward_stage_1 = -0.5e-3
bad_step_reward_stage_1 = -25 # общий
good_step_reward_stage_1 = 1
good_step_captured_reward_stage_1 = 2

**Тестирование модели**

In [8]:
game = Kalah()
episodes_count = 1000
neuronet_walker_queue = 1
count_win_neuronet = 0
draw_count = 0
count_choisen_zero = 0
bad_choise = []
rezult_step = "Хороший ход!"

och = []

for episode in range(episodes_count):
    game.set_new_game()
    if episode % 2:
        old_player_making_step = game.get_player_making_step()
        neuronet_walker_queue = 2
        while not game.get_game_over() and old_player_making_step == game.get_player_making_step():
            # bot_action = do_prof_bot_step(game.get_state())
            bot_action = do_simple_bot_step(game.get_state())
            # print("Бот делает ход, выбор", bot_action)
            rezult_step = game.take_step(bot_action)
            # game.print_state()
            # print("Ходит игрок ", game.get_player_making_step())
            # print("bot took step", game.get_player_making_step(), "Был выбор", bot_action, rezult_step)
    else:
        neuronet_walker_queue = 1

    while not game.get_game_over():

        # Выбор хода
        probs = model(game.get_state().to(torch.float))
        action = probs.argmax()

        while action in bad_choise:
            probs[probs.argmax()] = 0
            action = probs.argmax()
        # action = do_prof_bot_step(game.get_state())

        # Выбор награды и выполнение хода
        reward = 0
        old_state = game.get_state().to(torch.float)
        old_player_making_step = game.get_player_making_step()

        rezult_step = game.take_step(action + 1) # + 1)
        if rezult_step == "Куча пустая, выберите другую!":
            count_choisen_zero += 1
            # reward = bad_step_reward_stage_1

            print("Номер эпизода происшествия:", episode)
            # print("Выбор нейронки: ", action.data + 1)
            # print("Номер хода", neuronet_walker_queue, game.get_player_making_step())
            # game.print_state()
            bad_choise.append(action)
        else:
            # print("Neuronet took step", game.get_player_making_step(), "Был выбор", action + 1, rezult_step)
            # print("state:")
            # game.print_state()
            # print("Ходит игрок ", game.get_player_making_step())
            bad_choise = []
        # elif rezult_step == "Хороший ход!":
        #     reward = good_step_reward_stage_1
        # elif rezult_step == "Хороший ход! Захват!":
        #     reward = good_step_captured_reward_stage_1

        while not game.get_game_over() and old_player_making_step != game.get_player_making_step():
            bot_action = do_simple_bot_step(game.get_state())
            # print("Бот делает ход, выбор", bot_action)
            rezult_step = game.take_step(bot_action)
            # game.print_state()
            # print("Ходит игрок ", game.get_player_making_step())
            if rezult_step == "Куча пустая, выберите другую!":
                print("Bot took step", game.get_player_making_step(), "Был выбор", bot_action, rezult_step)
            # print("bot took step", game.get_player_making_step(), "Был выбор", bot_action, rezult_step)


    if game.get_winner() != None:
        # st = game.get_general_state()
        # # if episode%2:
        # #     print(torch.flip(st, [0]))
        # #     print(torch.flip(st, dims=(0,)))
        # och.append(st[:7].sum() -st[7:].sum())
        # count_win_neuronet += 1 if st[:7].sum() -st[7:].sum() > 0 else 0
        if game.get_winner() != 0:
            if neuronet_walker_queue == game.get_winner():
                count_win_neuronet += 1
        #         reward += winner_reward_stage_1
        #     else:
        #         reward += loser_reward_stage_1
        # else:
        #     reward += draw_reward_stage_1
        #     draw_count += 1
# OCH = torch.Tensor(och)
# print(f'Победы: {torch.where(OCH > 0, torch.ones(OCH.shape),torch.zeros(OCH.shape)).sum()}')

    # print("Очередь хода нейронки", neuronet_walker_queue)
    # print("Результат:", game.get_player_winner(), 'k', game.get_winner())
    # print("Номер попытки:", episode)
    # print("Награда/Штраф:", reward)

    # print(episode)

print(count_choisen_zero)
print(count_win_neuronet/episodes_count)
print(draw_count/episodes_count)

0
0.757
0.0


In [12]:
model

Sequential(
  (0): Linear(in_features=14, out_features=42, bias=True)
  (1): ReLU()
  (2): Linear(in_features=42, out_features=6, bias=True)
  (3): Softmax(dim=-1)
)

In [13]:
torch.save(model, "./l0l_test_1.pt")

In [9]:
# creating a 1D tensor
tens = torch.tensor([4, 5, 0, 25, 14])

# Display tensor
print("\n\nInput Tensor: ", tens)

# compute the element-wise entropy of
# input tensor
entr = torch.special.entr(tens).sum()

# Display result
print("\n\nComputed Entropy: ", entr)



Input Tensor:  tensor([ 4,  5,  0, 25, 14])


Computed Entropy:  tensor(-131.0111)
